In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Conv2D, Conv2DTranspose
from tensorflow.keras.layers import BatchNormalization, LeakyReLU
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.constraints import Constraint
from tensorflow.keras.datasets import cifar10  # CIFAR-10 데이터셋 예제로 사용
import numpy as np
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
import random
import tensorflow as tf


# GPU 디바이스 목록 가져오기

gpus = tf.config.experimental.list_physical_devices('GPU')

if gpus:
    print("사용 가능한 GPU 디바이스:")
    for gpu in gpus:
        print("디바이스 이름:", gpu.name)
else:
    print("사용 가능한 GPU 디바이스가 없습니다.")

In [ ]:
# 이미지 전처리 코드

# image_dir = "C:\\Users\\crazy\\Downloads\\archive\\img_align_celeba\\img_align_celeba"
# image_dir = "C:\\Users\\crazy\\Downloads\\archive\\img_align_celeba\\10000"
# image_dir = "C:\\Users\\crazy\\Downloads\\archive\\img_align_celeba\\10"
# image_dir = "C:\\Myfolder\\AI_tools\\dfl\\df_NVIDIA_RTX3000_series\\workspace\\data_src\\aligned"
# image_dir = "C:\\Myfolder\\AI_tools\\sd.webui\\this\\fun\\image"
# image_dir = "C:\\Users\\crazy\Downloads\\archive\\img_align_celeba\\facesetav"
image_dir = "C:\\Users\\crazy\Downloads\\archive\\img_align_celeba\\picted"
# crop_width, crop_height = 1024, 1024  # 자를 크기
resize_width, resize_height = 128, 128  # 변경할 크기

# 이미지 목록 가져오기
image_files = os.listdir(image_dir)

# 이미지를 NumPy 배열로 로드하고 크기 변경하기
images = []
for filename in image_files:
    img = tf.keras.preprocessing.image.load_img(os.path.join(image_dir, filename), target_size=(resize_width, resize_height))
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = img / 255.0  # 이미지 정규화
    images.append(img)
images = np.array(images)


# 이미지 중에서 무작위로 하나 선택
random_index = random.randint(0, len(images) - 1)
selected_image = images[random_index]

In [12]:
def build_generator():
    noise = Input(shape=(100,))
    x = Dense(4 * 4 * 512)(noise)
    x = Reshape((4, 4, 512))(x)
    x = Conv2DTranspose(256, kernel_size=5, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv2DTranspose(128, kernel_size=5, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv2DTranspose(3, kernel_size=5, strides=8, padding='same', activation='tanh')(x)
    generator = Model(noise, x)
    return generator

def build_discriminator():
    img_input = Input(shape=(128, 128, 3))
    x = Conv2D(64, kernel_size=5, strides=2, padding='same')(img_input)
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv2D(128, kernel_size=5, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv2D(256, kernel_size=5, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Flatten()(x)
    validity = Dense(100)(x)
    discriminator = Model(img_input, validity)
    return discriminator


In [13]:
class GradientPenalty(Constraint):
    def __init__(self, weight=10.0):
        self.weight = weight

    def __call__(self, w):
        batch_size = tf.shape(w)[0]
        epsilon = tf.random.uniform((batch_size, 1, 1, 1))
        interpolated = epsilon * w + (1 - epsilon) * (w + 0.5 * tf.random.normal(tf.shape(w)))
        gradient_penalty = self.weight * tf.reduce_sum(tf.square(interpolated - w))
        return gradient_penalty

mse_loss = MeanSquaredError()

def wasserstein_loss(y_true, y_pred):
    return -tf.reduce_mean(y_true * y_pred)

def gradient_penalty_loss(y_true, y_pred, averaged_samples):
    gradients = tf.gradients(y_pred, averaged_samples)[0]
    gradients_sqr = tf.square(gradients)
    gradients_sqr_sum = tf.reduce_sum(gradients_sqr, axis=np.arange(1, len(gradients_sqr.shape)))
    gradient_l2_norm = tf.sqrt(gradients_sqr_sum)
    gradient_penalty = tf.reduce_mean((gradient_l2_norm - 1) ** 2)
    return gradient_penalty


In [14]:
generator = build_generator()
discriminator = build_discriminator()

generator.summary()
discriminator.summary()

# 판별자의 가중치 제약을 추가합니다.
for layer in discriminator.layers:
    if isinstance(layer, Conv2D):
        layer.kernel_constraint = GradientPenalty()

real_img = Input(shape=(128, 128, 3))
fake_img = generator(Input(shape=(100,)))

validity_real = discriminator(real_img)
validity_fake = discriminator(fake_img)

discriminator_model = Model(inputs=[real_img, fake_img], outputs=[validity_real, validity_fake])

# 판별자와 생성자 모델의 컴파일
discriminator_model.compile(optimizer=RMSprop(0.00005), loss=[wasserstein_loss, wasserstein_loss],
                            loss_weights=[1, 1])
generator.compile(optimizer=RMSprop(0.00005), loss=wasserstein_loss)


Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_16 (InputLayer)       [(None, 100)]             0         
                                                                 
 dense_6 (Dense)             (None, 8192)              827392    
                                                                 
 reshape_3 (Reshape)         (None, 4, 4, 512)         0         
                                                                 
 conv2d_transpose_9 (Conv2DT  (None, 8, 8, 256)        3277056   
 ranspose)                                                       
                                                                 
 batch_normalization_12 (Bat  (None, 8, 8, 256)        1024      
 chNormalization)                                                
                                                                 
 leaky_re_lu_15 (LeakyReLU)  (None, 8, 8, 256)         0   

In [15]:
# 훈련 설정
epochs = 20000
batch_size = 64
sample_interval = 1000

for epoch in range(epochs):
    idx = np.random.randint(0, images.shape[0], batch_size)
    real_imgs = images[idx]

    noise = np.random.normal(0, 1, (batch_size, 100))
    fake_imgs = generator.predict(noise)

    d_loss_real = discriminator_model.train_on_batch([real_imgs, fake_imgs], [-np.ones((batch_size, 1)), np.ones((batch_size, 1))])
    
    noise = np.random.normal(0, 1, (batch_size, 100))
    valid = -np.ones((batch_size, 1))
    g_loss = generator.train_on_batch(noise, valid)

    if epoch % sample_interval == 0:
        print(f"Epoch {epoch}, D Loss Real: {d_loss_real[0]}, G Loss: {g_loss}")


2/2 [==============================] - 0s 32ms/step


ValueError: in user code:

    File "c:\Users\crazy\OneDrive\Desktop\Latent Diffusion\.conda\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\crazy\AppData\Local\Temp\ipykernel_16708\2536175825.py", line 15, in wasserstein_loss  *
        return -tf.reduce_mean(y_true * y_pred)

    ValueError: Dimensions must be equal, but are 64 and 128 for '{{node wasserstein_loss/mul}} = Mul[T=DT_FLOAT](IteratorGetNext:1, model_9/conv2d_transpose_11/Tanh)' with input shapes: [64,1], [64,128,128,3].
